In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
%config InlineBackend.figure_format = 'retina'

We looked at the name and details of every feature, and classfied them as three categories.

关于这个数据集，我仔细看了一下，

应该是每一行都有一个对应的customer和商品的信息的（见下customer_features和other_features）

从kaggle上看到“Food Mart (CFM) is a chain of convenience stores in the United States.”

所以可能是某个大公司旗下的各门店做media campaign，然后每行对应一个customer买product的信息，包括客户信息和商品promotion信息

所以这个cost可能指的是在这每一行信息的情况下，特定种类商品获取这个顾客的cost之类的


In [3]:
df = pd.read_csv("media prediction and its cost.csv")

In [4]:
# features基本上可以分为customer information和store information
# 为了简洁，和到时候网页上不用填那么多信息，我们这次就只用店铺信息
# 还能避免privacy和fairness之类的问题
# 不过这个选择只能人工手动辨别了
# 在项目展示的时候可以提一下
# 也提一下 我们因为去掉了很多信息，所以结果可能比较差
# 毕竟原本37个features的时候用random forest能达到R2=0.99
# 去掉很多features之后，还是60000多条data，留下的features里相似甚至相同的data可能就有完全不同的cost
# 我们可以回答说是这个project为了客户网页填写的简便，只保留了store features

# 应该是购买某商品的顾客信息
customer_features = ['marital_status','gender','total_children','education','member_card','occupation','houseowner',
                  'avg_cars_at home(approx)','avg. yearly_income','num_children_at_home']

# 店铺信息
store_features = ['store_sales(in millions)','store_cost(in millions)','unit_sales(in millions)',
                  'store_type','store_city','store_state','sales_country',
                  'store_sqft','grocery_sqft','frozen_sqft','meat_sqft','coffee_bar','video_store','salad_bar','florist']

# 应该是包括活动内容，活动特定商品的属性之类的
merchandise_features = ['promotion_name','food_category','food_department','food_family','brand_name','SRP','gross_weight','net_weight',
                  'recyclable_package','low_fat','units_per_case','media_type']

# target y
target = ['cost']

Too many features for a human customer to input on a webpage. So we do a feature selection by looking at their correlations.

In [5]:
# Get numerical features
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
num_df = df.select_dtypes(include=numerics)
num_features = list(num_df.columns)
print(num_df.shape)

(60428, 23)


In [6]:
# Categorical features
cat_df = df.select_dtypes(include="object")
cat_features = list(cat_df.columns)
print(cat_df.shape)

(60428, 17)


In [7]:
binary = []
for feature in num_features:
    different_values = len(df[feature].unique())
    #print((feature,different_values,different_values==2))
    if different_values == 2:
        binary.append(feature)
print(binary)

['recyclable_package', 'low_fat', 'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist']


In [9]:
data1=df[['cost','food_category','food_department','food_family','promotion_name','sales_country'
          ,'occupation','brand_name','store_type','store_city','store_state','media_type']]
data1.head(3)

,cost,food_category,food_department,food_family,promotion_name,sales_country,occupation,brand_name,store_type,store_city,store_state,media_type
0,126.62,Breakfast Foods,Frozen Foods,Food,Bag Stuffers,USA,Skilled Manual,Carrington,Deluxe Supermarket,Salem,OR,"Daily Paper, Radio"
1,59.86,Breakfast Foods,Frozen Foods,Food,Cash Register Lottery,USA,Professional,Carrington,Deluxe Supermarket,Salem,OR,"Daily Paper, Radio"
2,84.16,Breakfast Foods,Frozen Foods,Food,High Roller Savings,USA,Manual,Carrington,Deluxe Supermarket,Salem,OR,"Daily Paper, Radio"


Calculate the correlations between cost and unordered multivariate categorical variables, using ANOVA method

In [10]:
import scipy.stats as stats
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
model=ols('cost~food_category+food_department+food_family+promotion_name+sales_country+occupation+brand_name+store_type+store_city+store_state+media_type'
          ,data=data1).fit()
anova_table=anova_lm(model,typ=2)
print('check the p value')
print(anova_table)

check the p value
                       sum_sq       df           F    PR(>F)
food_category    2.570261e+04     44.0    1.001959  0.390728
food_department  1.226715e+04     21.0    1.001959  0.390728
food_family      1.485538e+02      2.0    0.127403  0.721141
promotion_name   1.246315e+07     48.0  445.360928  0.000000
sales_country    1.625746e+03      2.0    1.394274  0.248021
occupation       3.098347e+03      4.0    1.328604  0.256544
brand_name       6.425652e+04    110.0    1.001959  0.390728
store_type       2.336601e+03      4.0    1.001959  0.390728
store_city       1.051470e+04     18.0    1.001959  0.390728
store_state      5.257352e+03      9.0    1.001959  0.390728
media_type       2.899953e+06     12.0  414.510146  0.000000
Residual         3.509767e+07  60201.0         NaN       NaN


C:\Users\ylwu5\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 44, but rank is 3
  warnings.warn('covariance of constraints does not have full '
C:\Users\ylwu5\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 21, but rank is 3
  warnings.warn('covariance of constraints does not have full '
C:\Users\ylwu5\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '
C:\Users\ylwu5\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 110, but rank is 3
  warnings.warn('covariance of constraints does not have ful

From the p-values we can conclude that among unordered variables, promotion name and media type is related with cost, whose p-values are far smaller than 0.05.

Then we are going to check the correlations between binary variables and cost, the usually way is to calculate the point-biserial correlation.

Before calculation, we encode the character binary variable into 0 and 1.


In [11]:
marital=["M","S"]
dummy_vars=[1,0]
df['maritaldummy']=df['marital_status'].replace(marital,dummy_vars)
stats.pointbiserialr(df["cost"],df['maritaldummy'])

PointbiserialrResult(correlation=-0.0007011651080323834, pvalue=0.8631563917541039)

In [12]:
genders=["F","M"]
dummy_vars=[1,0]
df['genderdummy']=df['gender'].replace(genders,dummy_vars)
stats.pointbiserialr(df["cost"],df['genderdummy'])

PointbiserialrResult(correlation=0.008603374408391285, pvalue=0.03443965471443298)

In [13]:
house_owners=["Y","N"]
dummy_vars=[0,1]
df['housedummy']=df['houseowner'].replace(house_owners,dummy_vars)
stats.pointbiserialr(df["cost"],df['housedummy'])

PointbiserialrResult(correlation=-0.0033878574659527796, pvalue=0.4049620924061012)

We can conclude that the gender is related with the cost because the p-value is smaller than 0.05.

Then calculate the correlations of the remaining unordered binary variables.

In [16]:
bidata=df[{'recyclable_package','low_fat','coffee_bar','video_store','prepared_food','florist'}]
for col in bidata.columns:
    print(f"The result between cost and {col} is:\n{stats.pointbiserialr(df['cost'],df[col])}")

The result between cost and florist is:
PointbiserialrResult(correlation=-0.08734049540628466, pvalue=1.2334543558349544e-102)
The result between cost and prepared_food is:
PointbiserialrResult(correlation=-0.09559609699909873, pvalue=1.1694696744399297e-122)
The result between cost and coffee_bar is:
PointbiserialrResult(correlation=-0.06085850311031529, pvalue=1.087964797206913e-50)
The result between cost and recyclable_package is:
PointbiserialrResult(correlation=-0.0017198840904629285, pvalue=0.6724591960483609)
The result between cost and video_store is:
PointbiserialrResult(correlation=-0.11121393282119389, pvalue=1.4480785136680184e-165)
The result between cost and low_fat is:
PointbiserialrResult(correlation=0.004207019183325482, pvalue=0.301062700974472)


In conclusion, we can get that:
1.  There is no evidence showing that there are no relationship between the cost and recyclable_package or low_fat, because the p-values above are bigger than 0.05;
2.  We can conclude that the coffee_bar, video_store, salad_bar, prepared_food, and florist are related with the cost because the p-values above are smaller than 0.05, although the correlations are not large.


In [18]:
data2=df[['cost','store_sales(in millions)','store_cost(in millions)','SRP',
          'gross_weight','net_weight','store_sqft','grocery_sqft','frozen_sqft',
          'meat_sqft']]
data2.corr(method='kendall')['cost']

cost                        1.000000
store_sales(in millions)   -0.004897
store_cost(in millions)    -0.004955
SRP                         0.001289
gross_weight                0.001023
net_weight                  0.001012
store_sqft                 -0.028274
grocery_sqft               -0.018307
frozen_sqft                -0.053951
meat_sqft                  -0.053951
Name: cost, dtype: float64

From these,if take 0.1 as threshold, it shows that cost is not related with store_sales, store_cost, SRP, gross_weight, net_weight, store_sqft， and grocery_sqft.

Finally, we calculate the correlation about ordered features by using spearman correlation.

We recode the string variables first, then calculate the correlations.


In [19]:
education=["Partial High School","High School Degree","Partial College","Bachelors Degree","Graduate Degree"]
dummy_vars=[0,1,2,3,4]
df['educationdummy']=df['education'].replace(education,dummy_vars)

member_card=["Normal","Silver","Bronze","Golden"]
dummy_vars=[0,1,2,3]
df['memberdummy']=df['member_card'].replace(member_card,dummy_vars)

income=["$10K - $30K","$30K - $50K","$50K - $70K","$70K - $90K","90K - $110K","$110K - $130K","$130K - $150K","$150K +"]
dummy_vars=[0,1,2,3,4,5,6,7]
df['incomedummy']=df['avg. yearly_income'].replace(income,dummy_vars)


In [20]:
data3=df[['cost','unit_sales(in millions)','total_children','avg_cars_at home(approx)',
          'num_children_at_home','educationdummy','memberdummy','incomedummy','units_per_case']]
data3.corr(method='spearman')['cost']

cost                        1.000000
unit_sales(in millions)    -0.014185
total_children             -0.003614
avg_cars_at home(approx)    0.009466
num_children_at_home       -0.003305
educationdummy              0.004307
memberdummy                -0.003334
units_per_case             -0.002649
Name: cost, dtype: float64

After calculation, if taking 0.1 as threshold, we find that they are all not related with the cost.

In conclusion, after calculating the correlation by using different methods based on the different types of features, these are the features most related to the cost:

promotion_name, gender, frozen_sqft, meat_sqft, coffee_bar, video_store, salad_bar, prepared_food, florist, media_type

In [21]:
print(f"Number of unique values in {df['cost'].shape[0]} data:")
for col in df.columns:
    print(f"{col}: {len(df[col].unique())}")

Number of unique values in 60428 data:
food_category: 45
food_department: 22
food_family: 3
store_sales(in millions): 1033
store_cost(in millions): 9919
unit_sales(in millions): 6
promotion_name: 49
sales_country: 3
marital_status: 2
gender: 2
total_children: 6
education: 5
member_card: 4
occupation: 5
houseowner: 2
avg_cars_at home(approx): 5
avg. yearly_income: 8
num_children_at_home: 6
avg_cars_at home(approx).1: 5
brand_name: 111
SRP: 315
gross_weight: 376
net_weight: 332
recyclable_package: 2
low_fat: 2
units_per_case: 36
store_type: 5
store_city: 19
store_state: 10
store_sqft: 20
grocery_sqft: 20
frozen_sqft: 20
meat_sqft: 20
coffee_bar: 2
video_store: 2
salad_bar: 2
prepared_food: 2
florist: 2
media_type: 13
cost: 328
maritaldummy: 2
genderdummy: 2
housedummy: 2
educationdummy: 5
memberdummy: 4
incomedummy: 8


We can see that there are only 328 different values of cost in 60428 rows of data. Also for other numerical feature they have much less number unique values than 60428. So by Random Forest model and using up all features, we fit with complex condition judgements in the model and build a regression model that is almost like a classification one.